In [79]:
# ライブラリのインポート
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import os
import zipfile
import matplotlib.pyplot as plt
import japanize_matplotlib 
from matplotlib.ticker import MaxNLocator
from tqdm.auto import tqdm
from shapely.geometry import LineString, Point
import datetime as dt
from pathlib import Path
import pydeck as pdk
import numpy as np
import math
from geopy.distance import geodesic

from shapely.wkt import loads

import gurobipy as gu
from gurobipy import Model, GRB
import logging
from itertools import product


In [41]:
# --- 定数定義 ---
# パス設定
PROJECT_ROOT = "../"
DATA_DIR = PROJECT_ROOT + 'data/'
OUTPUT_DIR = PROJECT_ROOT + 'output/'
INTERMEDIATE_OUTPUT_DIR = OUTPUT_DIR + 'intermediate_data/'

# 入力データパス
AGRI_DIR = DATA_DIR + 'agricultural_settlements/'
DEM_DIR = DATA_DIR  + 'DEM250/'
PGV_DIR = DATA_DIR + 'jshis_pgv/'
POP_DIR = DATA_DIR + 'population/'
MESH_DIR = DATA_DIR + 'mesh/'

# 座標参照系 (CRS)
TARGET_CRS = "EPSG:4612"
WGS84_CRS = "EPSG:4326" # 緯度経度計算用のCRS

# 1. データの読み込み

In [42]:
def load_shapefiles_from_dir(directory: str, target_crs: str) -> gpd.GeoDataFrame:
    """指定ディレクトリ以下の全シェープファイルを再帰的に読み込み、結合して返す。"""
    directory = Path(directory)
    logging.info(f"Loading shapefiles from {directory}...")
    shp_files = list(directory.rglob('*.shp'))
    if not shp_files:
        logging.warning(f"No shapefiles found in {directory}.")
        return gpd.GeoDataFrame()
    gdfs = [gpd.read_file(f) for f in shp_files]
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    if combined_gdf.crs is None:
        combined_gdf.set_crs(target_crs, inplace=True)
    return combined_gdf.to_crs(target_crs)

In [43]:
def load_population_data(pop_dir: str, mesh_dir: str, target_crs: str) -> gpd.GeoDataFrame:
    """人口統計テキストデータとメッシュシェープファイルを読み込み、結合してGeoDataFrameを返す。"""
    pop_dir = Path(pop_dir)
    logging.info(f"Loading population data from {pop_dir}...")
    txt_files = list(pop_dir.rglob('*.txt'))
    if not txt_files:
        logging.warning(f"No population text files found in {pop_dir}.")
        return gpd.GeoDataFrame()
    df_col = pd.read_csv(txt_files[0], encoding="sjis", nrows=1)
    col1 = df_col.columns.tolist()
    col2 = df_col.values[0].tolist()
    columns = [c1 if pd.isna(c2) else c2.replace("\u3000", "_").lstrip("_") for c1, c2 in zip(col1, col2)]
    pop_dfs = [pd.read_csv(f, encoding="sjis", header=None, skiprows=2, names=columns) for f in txt_files]
    pop_df = pd.concat(pop_dfs, ignore_index=True)
    pop_df['KEY_CODE'] = pop_df['KEY_CODE'].astype(str)
    mesh_gdf = load_shapefiles_from_dir(Path(mesh_dir), target_crs)
    pop_gdf = mesh_gdf.merge(pop_df[['KEY_CODE', '人口（総数）']], on="KEY_CODE")
    return gpd.GeoDataFrame(pop_gdf, geometry='geometry', crs=target_crs)

In [44]:
agri_gdf_raw = load_shapefiles_from_dir(AGRI_DIR, TARGET_CRS)
dem_gdf_raw = load_shapefiles_from_dir(DEM_DIR, TARGET_CRS)
pgv_gdf_raw = load_shapefiles_from_dir(PGV_DIR, TARGET_CRS)
pop_gdf = load_population_data(POP_DIR, MESH_DIR, TARGET_CRS)

# 2. 分析エリアの設定とデータのクリッピング

In [45]:
def define_analysis_area(gdf: gpd.GeoDataFrame, city_name: str, buffer_distance: float) -> dict:
    logging.info(f"Defining analysis area based on city: '{city_name}'")
    area_gdf = gdf[gdf["CITY_NAME"].str.contains(city_name, na=False)].copy()
    if area_gdf.empty:
        raise ValueError(f"No geometries found for city '{city_name}'")
    target_geometry = area_gdf.union_all()
    buffered_geometry = target_geometry.buffer(buffer_distance)
    return {"filtered_gdf": area_gdf, "target_geometry": target_geometry, "buffered_geometry": buffered_geometry}

In [46]:
def clip_data_to_area(gdf: gpd.GeoDataFrame, clip_geometry) -> gpd.GeoDataFrame:
    return gdf[gdf.intersects(clip_geometry)].copy()

In [ ]:
def get_road_network(agri_gdf, crs: str) -> gpd.GeoDataFrame:
    logging.info("Fetching road network from OSMnx...")
    # キャッシュは有効にしておく（デフォルトはTrue）。OSMnx側でupdateされた時に最新のデータを取得するにはFalseにする。
    ox.settings.use_cache = True 
    polygon_wgs84 = gpd.GeoSeries([agri_gdf.union_all()], crs=crs).to_crs(WGS84_CRS).iloc[0]
    graph = ox.graph_from_polygon(polygon_wgs84.envelope, network_type='drive')
    _, edges_gdf = ox.graph_to_gdfs(graph)
    edges_gdf = edges_gdf[edges_gdf["geometry"].intersects(agri_gdf.unary_union)]
    return edges_gdf.to_crs(crs)

In [48]:
analysis_area = define_analysis_area(gdf=agri_gdf_raw, city_name='常陸', buffer_distance=0.05)
agri_gdf = analysis_area["filtered_gdf"]
clip_geom = analysis_area["buffered_geometry"]
dem_gdf = clip_data_to_area(dem_gdf_raw, clip_geom)
pgv_gdf = clip_data_to_area(pgv_gdf_raw, clip_geom)
road_gdf = get_road_network(agri_gdf, TARGET_CRS)

/var/folders/dr/zxykd_7d3519k_l3xpcbrg4r0000gn/T/ipykernel_92711/392682960.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  edges_gdf = edges_gdf[edges_gdf["geometry"].intersects(agri_gdf.unary_union)]


In [49]:
# 出力ファイルパス
AGRI_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'agri_gdf.pkl'
DEM_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'dem_gdf.pkl'
PGV_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'pgv_gdf.pkl'
ROAD_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'road_gdf.pkl'

In [50]:
# GeoDataFrameの保存 (Pickle形式)
agri_gdf.to_pickle(AGRI_GDF_PKL)
dem_gdf.to_pickle(DEM_GDF_PKL)
pgv_gdf.to_pickle(PGV_GDF_PKL)
road_gdf.to_pickle(ROAD_GDF_PKL)

# 3. リスク計算と人口集計

In [51]:
# モデルパラメータ (論文のTable 3より)
BETA_0 = -6.3
BETA_1 = 1.65
BETA_2 = 0.06
BETA_3 = 0.01

In [52]:
def calculate_landslide_risk(dem_gdf: gpd.GeoDataFrame, pgv_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    logging.info("Calculating landslide risk for each terrain mesh...")
    dem_with_pgv = pd.merge(pgv_gdf[['CODE', 'T50_P39_SV', 'geometry']], dem_gdf[['G04d_001', 'G04d_010_numeric']], left_on='CODE', right_on='G04d_001', how='left').drop(columns=['G04d_001'])
    dem_with_pgv.rename(columns={'T50_P39_SV': 'pgv', 'G04d_010_numeric': 'slope'}, inplace=True)
    logit_prob = (BETA_0 + BETA_1 * np.log(dem_with_pgv['pgv'].clip(lower=1e-9)) + BETA_2 * dem_with_pgv['slope'] + BETA_3 * np.log(dem_with_pgv['pgv'].clip(lower=1e-9)) * dem_with_pgv['slope'])
    dem_with_pgv['landslide_prob'] = 1 / (1 + np.exp(-logit_prob))
    return dem_with_pgv

In [53]:
def calculate_isolation_risk(agri_gdf: gpd.GeoDataFrame, road_gdf: gpd.GeoDataFrame, risk_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    logging.info("Calculating road disruption and settlement isolation risk...")
    road_with_risk = gpd.sjoin(road_gdf[['geometry']], risk_gdf[['landslide_prob', 'geometry']], how="left", predicate="intersects")
    road_disruption_prob = 1 - road_with_risk.groupby('geometry')['landslide_prob'].apply(lambda p: (1 - p).prod())
    road_disruption_prob.name = 'disruption_prob'
    road_with_prob = road_gdf.join(road_disruption_prob, on='geometry')
    agri_boundaries = agri_gdf.copy()
    agri_boundaries['geometry'] = agri_boundaries.boundary
    settlement_road_join = gpd.sjoin(agri_boundaries, road_with_prob[['disruption_prob', 'geometry']], how='left', predicate="intersects")
    isolation_prob = settlement_road_join.groupby(settlement_road_join.index)['disruption_prob'].prod()
    isolation_prob.name = 'isolation_prob'
    syuraku_with_isolation = agri_gdf.join(isolation_prob, how='left')
    mean_prob = syuraku_with_isolation['isolation_prob'].mean()
    syuraku_with_isolation['isolation_prob'].fillna(value=mean_prob, inplace=True)
    return syuraku_with_isolation

In [ ]:
def aggregate_population_by_settlement(syuraku_with_isolation_gdf: gpd.GeoDataFrame, pop_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    logging.info("Aggregating population and calculating expected victims...")
    syuraku_with_pop = gpd.sjoin(syuraku_with_isolation_gdf, pop_gdf[['人口（総数）', 'KEY_CODE', 'geometry']], how='left', predicate="intersects")
    syuraku_with_pop.drop_duplicates(subset='KEY_CODE', keep='first', inplace=True)
    pop_sum = syuraku_with_pop.groupby(syuraku_with_pop.index).agg(total_population=('人口（総数）', 'sum'))
    result_gdf = syuraku_with_isolation_gdf.join(pop_sum, how='left')
    result_gdf['total_population'].fillna(0, inplace=True)
    result_gdf['expected_victims'] = result_gdf['total_population'] * result_gdf['isolation_prob']
    return result_gdf.reset_index(drop=True).reset_index().rename(columns={'index': 'syuraku_id'})

In [55]:
dem_gdf['G04d_010_numeric'] = pd.to_numeric(dem_gdf['G04d_010'], errors='coerce')
risk_gdf = calculate_landslide_risk(dem_gdf, pgv_gdf)
syuraku_with_isolation_gdf = calculate_isolation_risk(agri_gdf, road_gdf, risk_gdf)
syuraku_pop_risk_gdf = aggregate_population_by_settlement(syuraku_with_isolation_gdf, pop_gdf)

/var/folders/dr/zxykd_7d3519k_l3xpcbrg4r0000gn/T/ipykernel_92711/1875742036.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  syuraku_with_isolation['isolation_prob'].fillna(value=mean_prob, inplace=True)
/var/folders/dr/zxykd_7d3519k_l3xpcbrg4r0000gn/T/ipykernel_92711/3369453510.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on w

In [ ]:
# 出力ファイルパス
RISK_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'risk_gdf.pkl'
SYURAKU_WITH_ISOLATION_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'syuraku_with_isolation_gdf.pkl'
SYURAKU_POP_RISK_GDF_PKL = INTERMEDIATE_OUTPUT_DIR + 'syuraku_pop_risk_gdf.pkl'
# GeoDataFrameの保存 (Pickle形式)
risk_gdf.to_pickle(RISK_GDF_PKL)
syuraku_with_isolation_gdf.to_pickle(SYURAKU_WITH_ISOLATION_GDF_PKL)
syuraku_pop_risk_gdf.to_pickle(SYURAKU_POP_RISK_GDF_PKL)

# 4. 配送先と施設候補地の準備

In [73]:
def prepare_destination_and_facility_nodes(agri_gdf: gpd.GeoDataFrame, syuraku_pop_risk_gdf: gpd.GeoDataFrame) -> tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    logging.info("Preparing destination and facility nodes...")
    destination_nodes = syuraku_pop_risk_gdf.copy()
    destination_nodes['geometry'] = destination_nodes.geometry.centroid
    analysis_area_polygon = agri_gdf.union_all()
    bounding_polygon = analysis_area_polygon.envelope
    graph = ox.graph_from_polygon(bounding_polygon, network_type='drive')
    nodes_gdf, _ = ox.graph_to_gdfs(graph)
    nodes_gdf = nodes_gdf.to_crs(TARGET_CRS)
    facility_candidates = nodes_gdf[(nodes_gdf.street_count >= 4) & (nodes_gdf.intersects(analysis_area_polygon))].copy()
    facility_nodes = facility_candidates.reset_index().reset_index().rename(columns={'index': 'facility_id'})
    return destination_nodes, facility_nodes


In [74]:
destination_nodes, facility_nodes = prepare_destination_and_facility_nodes(agri_gdf, syuraku_pop_risk_gdf)

/var/folders/dr/zxykd_7d3519k_l3xpcbrg4r0000gn/T/ipykernel_92711/754827305.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  destination_nodes['geometry'] = destination_nodes.geometry.centroid


In [75]:
# 出力ファイルパス
DESTINATION_NODES_PKL = INTERMEDIATE_OUTPUT_DIR + 'destination_nodes.pkl'
FACILITY_NODES_PKL = INTERMEDIATE_OUTPUT_DIR + 'facility_nodes.pkl'
# GeoDataFrameの保存 (Pickle形式)
destination_nodes.to_pickle(DESTINATION_NODES_PKL)
facility_nodes.to_pickle(FACILITY_NODES_PKL)

# 5.最適化用データ作成

In [87]:
def prepare_optimization_variables(destination_nodes: gpd.GeoDataFrame, facility_nodes: gpd.GeoDataFrame) -> dict:
    """最適化モデルに必要なパラメータ、セット、行列を準備する。"""
    logging.info("Preparing variables for optimization...")

    # --- パラメータ定義 ---
    NUM_OPEN_FACILITIES = 12       # k: 開設される施設の数
    UAV_ENDURANCE = 3              # E: 各UAVの耐久距離 (km)
    EXCHANGE_RATE = 164.48         # 1ユーロあたりの円
    ITEM_PRICE_MARGIN = 20 * EXCHANGE_RATE
    FIX_COST = 1 * EXCHANGE_RATE
    MAX_DELIVERY_COST = 1 * EXCHANGE_RATE

    # --- セット定義 ---
    # Gurobiで扱いやすいように、0から始まる連続した整数IDにマッピングする
    facility_map = {fid: i for i, fid in enumerate(facility_nodes['facility_id'])}
    destination_map = {did: i for i, did in enumerate(destination_nodes['syuraku_id'])}

    NODE_F = np.array(list(facility_map.values()))
    NODE_D = np.array(list(destination_map.values()))

    # --- 距離行列と収益性行列の計算 ---
    logging.info("Calculating distance and profitability matrices...")

    # 緯度経度(WGS84)に変換して座標を取得
    fac_coords = facility_nodes.to_crs(WGS84_CRS)['geometry'].apply(lambda p: (p.y, p.x))
    dest_coords = destination_nodes.to_crs(WGS84_CRS)['geometry'].apply(lambda p: (p.y, p.x))

    # 全ての組み合わせの距離を効率的に計算
    # itertools.productで全組み合わせを生成し、geodesicで距離(km)を計算
    all_distances = [geodesic(p1, p2).km for p1, p2 in product(fac_coords, dest_coords)]
    DISTANCE_F_D = np.array(all_distances).reshape(len(NODE_F), len(NODE_D))

    # 収益性計算の準備
    # 人口データを需要地点の順序に合わせてNumpy配列に変換
    pop_d_array = destination_nodes['total_population'].to_numpy()
    
    # ブロードキャストを利用して収益性行列を一括計算
    # (1, N_NODE_D)の形状の人口配列と(N_NODE_F, N_NODE_D)の形状の距離行列で計算
    profit_per_person = ITEM_PRICE_MARGIN - FIX_COST - (MAX_DELIVERY_COST * DISTANCE_F_D / UAV_ENDURANCE)
    PROFITABILITY_F_D = pop_d_array[np.newaxis, :] * profit_per_person
    
    # --- その他の配列定義 ---
    POPULATION_D = pop_d_array
    RISK_D = destination_nodes['isolation_prob'].to_numpy()

    # --- 目的関数の定数 ---
    MAX_SUM_UNCOVERED_ISOLATED_PEOPLE = np.sum(RISK_D * POPULATION_D)

    # 人口が0の地点に微小なボーナス(epsilon)を与え、それ以外は0にする
    epsilon = 1e-4  # 主目的を邪魔しない非常に小さい値
    COVERAGE_BONUS_D = np.where(POPULATION_D == 0, epsilon, 0)
    logging.info(f"Created coverage bonus for {np.sum(COVERAGE_BONUS_D > 0)} zero-population points.")
    # ★★★ 変更点 End ★★★
    
    logging.info("Finished preparing optimization variables.")

    # --- 結果を辞書にまとめる ---
    optimization_vars = {
        "DISTANCE_F_D": DISTANCE_F_D,
        "PROFITABILITY_F_D": PROFITABILITY_F_D,
        "POPULATION_D": POPULATION_D,
        "RISK_D": RISK_D,
        "NODE_F": NODE_F,
        "NODE_D": NODE_D,
        "facility_map": facility_map,
        "destination_map": destination_map,
        "NUM_OPEN_FACILITIES": NUM_OPEN_FACILITIES,
        "MAX_SUM_UNCOVERED_ISOLATED_PEOPLE": MAX_SUM_UNCOVERED_ISOLATED_PEOPLE,
        "COVERAGE_BONUS_D": COVERAGE_BONUS_D, # ★★★ 変更点: 保存対象に追加 ★★★
        "UAV_ENDURANCE": UAV_ENDURANCE # ★★★ 変更点: UAV_ENDURANCEも保存 ★★★

    }
    return optimization_vars

In [88]:
optimization_vars = prepare_optimization_variables(destination_nodes, facility_nodes)

In [89]:
OPTIMIZATION_VARS_NPZ = INTERMEDIATE_OUTPUT_DIR + 'optimization_vars.npz'

In [90]:
np.savez(OPTIMIZATION_VARS_NPZ, **optimization_vars)

In [84]:
optimization_vars["PROFITABILITY_F_D"]

array([[2296375.08641333, 3173258.24365355,  529719.77970308, ...,
          62561.82384974,  263267.12291465,  326305.84367958],
       [2318842.31914598, 3203236.61054858,  534871.14553004, ...,
          63457.02333506,  267113.76512403,  331306.50967859],
       [2818413.88377875, 4035094.3542033 ,  624721.56401955, ...,
          90592.39536386,  402102.37194163,  516857.58169153],
       ...,
       [2987224.0976406 , 4509691.86995602,  651617.09657289, ...,
          76807.67936074,  341047.82306867,  435356.95871823],
       [2987127.87206138, 4523685.10457108,  651427.31026803, ...,
          76145.70129891,  338091.858735  ,  431385.16187292],
       [2987778.39024266, 4526950.99097405,  651550.15970739, ...,
          76042.66060754,  337621.42475535,  430740.57325526]],
      shape=(843, 332))